In [5]:
from models import TimeSeriesTransformer
from dataset import TransformerDataset
from torch.utils.data import DataLoader
from utils import *
import os


In [18]:
path = './Datasets/intracardiac_dataset/'
VmData, pECGData = fileReader(path)
print('Data loading from files - complete')
## Model parameters
dim_val = 512 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 8 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_decoder_layers = 4 # Number of times the decoder layer is stacked in the decoder
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
input_size = 12 # The number of input variables. 1 if univariate forecasting.
dec_seq_len = 92 # length of input given to decoder. Can have any integer value.
enc_seq_len = 153 # length of input given to encoder. Can have any integer value.
output_sequence_length = 58 # Length of the target sequence, i.e. how many time steps should your forecast cover
max_seq_len = enc_seq_len # What's the longest sequence the model will encounter? Used to make the positional encoder
batch_size = 128
batch_first= False

Loading datafiles :  78%|███████▊  | 12520/16117 [00:46<00:13, 267.97it/s]


KeyboardInterrupt: 

In [13]:
trainpECGData = pECGData[:int(0.8*pECGData.shape[0])]
testpECGData = pECGData[int(0.8*pECGData.shape[0]):]

trainVMdata = VmData[:int(0.8*VmData.shape[0])]
testVMdata = VmData[int(0.8*pECGData.shape[0]):]

In [15]:
# Get the indices of the sequences
# The idea is: start - stop, where stop - start is window_size
# This means, each tuple in VmInd and pECGInd is 50 steps
TrainVmInd, TrainpECGInd = get_indices_entire_sequence(VmData = trainVMdata, 
                                            ECGData = trainpECGData, 
                                            window_size= enc_seq_len + output_sequence_length, 
                                            step_size = 1)

# Now let's collect the training data in the Transformer Dataset class
training_data = TransformerDataset(VmData = trainVMdata,
                                    ECGData = trainpECGData,
                                    VmIndices= TrainVmInd,
                                    ECGInd = TrainpECGInd,
                                    enc_seq_len = enc_seq_len,
                                    dec_seq_len = dec_seq_len,
                                    target_seq_len = output_sequence_length)

TestVmInd, TestpECGInd = get_indices_entire_sequence(VmData = testVMdata, 
                                            ECGData = testpECGData, 
                                            window_size= enc_seq_len + output_sequence_length, 
                                            step_size = 1)

# Now let's collect the training data in the Transformer Dataset class
testing_data = TransformerDataset(VmData = testVMdata,
                                    ECGData = testpECGData,
                                    VmIndices= TestVmInd,
                                    ECGInd = TestpECGInd,
                                    enc_seq_len = enc_seq_len,
                                    dec_seq_len = dec_seq_len,
                                    target_seq_len = output_sequence_length)



trainLoader = DataLoader(training_data, batch_size)
testLoader = DataLoader(testing_data, batch_size)

In [17]:
if not batch_first:
    # Permute from shape [batch size, seq len, num features] to [seq len, batch size, num features]
    

    model = TimeSeriesTransformer(
    dim_val=dim_val,
    batch_first=batch_first,
    input_size=input_size, 
    dec_seq_len=dec_seq_len,
    out_seq_len=output_sequence_length, 
    n_decoder_layers=n_decoder_layers,
    n_encoder_layers=n_encoder_layers,
    n_heads=n_heads)

    

    for src, trg, trg_y in tqdm(trainLoader, desc='Training:  '):
        src_mask = generate_square_subsequent_mask(
        dim1=output_sequence_length,
        dim2=enc_seq_len
        )

        tgt_mask = generate_square_subsequent_mask( 
        dim1=output_sequence_length,
        dim2=output_sequence_length
        )
        src = src.permute(1, 0, 2)
        trg = trg.permute(1, 0, 2)
        output = model(
        src=src.float(),
        tgt=trg.float(),
        src_mask=src_mask.float(),
        tgt_mask=tgt_mask.float()
        )


Training:  :   0%|          | 0/3 [00:12<?, ?it/s]


KeyboardInterrupt: 